In [28]:
from tqdm import tqdm_notebook as tqdm

In [29]:
import requests, json

In [66]:
dfs = []
years = [2010, 2011, 2012, 2014, 2016]

In [67]:
for year in tqdm(years):
    try:
        url = "https://api.census.gov/data/{}/acs/acs5/profile?get=DP02_0067PE,NAME&for=congressional%20district:*".format(year)
        df = pd.read_json((requests.get(url).content))
        df.columns = df.iloc[0].values
        df = df.iloc[1:].copy()
        df['year'] = year
        dfs.append(df)
    except:
        pass

In [68]:
df = pd.concat(dfs)

In [69]:
df.columns = ['percent_bachelor_or_above', 'name', 'state_fips', 'district', 'year']

In [113]:
df1 = pd.read_csv('ACS_06_EST_S1501/ACS_06_EST_S1501_with_ann.csv')

col = ['HC01_EST_VC05', 'GEO.id2', 'GEO.display-label']

df1 = df1[col].iloc[1:].copy()
df1['year'] = 2006

df2 = pd.read_csv('ACS_08_3YR_S1501/ACS_08_3YR_S1501_with_ann.csv')

df2 = df2[col].iloc[1:].copy()
df2['year'] = 2008

df3 = pd.read_csv('DEC_00_SF3_DP2/DEC_00_SF3_DP2_with_ann.csv')

col = ['HC01_VC18', 'GEO.id2', 'GEO.display-label']
df3 = df3[col].iloc[1:].copy()
df3['year'] = 2000

col += ['year']
df1.columns = col
df2.columns = col
df3.columns = col

In [114]:
df_other = pd.concat([df1, df2, df3])

In [115]:
df_other['state_fips'] = df_other['GEO.id2'].apply(lambda x : x[:2])
df_other['district'] = df_other['GEO.id2'].apply(lambda x : x[2:])

In [116]:
df_other = df_other.drop('GEO.id2', axis=1)

In [117]:
df_other.columns = ['percent_bachelor_or_above', 'name', 'year', 'state_fips', 'district']

In [119]:
df = pd.concat([df, df_other], sort=False)

In [135]:
df['name'].str.findall(r'\(([^\)]+?th.+?)\)').apply(lambda x : x[0]).value_counts()

110th Congress    437
115th Congress    437
111th Congress    437
113th Congress    437
112th Congress    437
109th Congress    437
114th Congress    437
106th Congress    436
Name: name, dtype: int64

In [126]:
df.reset_index().to_feather('percent_bachelor_or_above.feather')